In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import json
import random
import numpy as np

2026-01-15 19:07:10.405397: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


TensorFlow version: 2.17.1
Num GPUs Available:  1


2026-01-15 19:07:12.197453: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-01-15 19:07:12.197502: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-01-15 19:07:12.592085: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-01-15 19:07:12.592130: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2026-01-15 19:07:12.592152: I external/local_xla/xla/stream_executor/rocm/rocm_executor.

## Priprema podataka

Iz json datoteka potrebno je izvući ono bitno, a to su dvije liste potrebna za ubaciti u mrežu koju treniramo:
+ lista puteva do slika
+ lista koja označava kojoj kategoriji odgovara pojedina slika

In [3]:
PATH_TO_JSON_FILE = "../fastener_dataset/annotations/instances_default.json"
PATH_TO_IMAGES_FOLDER = "../fastener_dataset/images"

In [4]:
with open(PATH_TO_JSON_FILE, "r") as f:
    data = json.load(f)

images = data["images"]
print("Number of images: ",len(images))
print("Attributes: ",list(images[0].keys()))
print()

annotations = data["annotations"]
print("Number of annotations (they refer to images):",len(annotations))
print("Attributes: ", list(annotations[0].keys()))
print()
last_annotation = annotations[-1]
print("IDs don't always match!")
print(last_annotation["id"],"!=",last_annotation["image_id"])

Number of images:  16970
Attributes:  ['id', 'width', 'height', 'file_name', 'license', 'flickr_url', 'coco_url', 'date_captured']

Number of annotations (they refer to images): 16970
Attributes:  ['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd', 'attributes']

IDs don't always match!
17011 != 17000


Bitno mi je samo images.file_name, annotations.category_id i annotations.category (za model s podklasama)

ID ne prati indeksni broj zapisa!

In [5]:
paths = []
labels = []
labels_for_subclasses = []

for annotation in annotations:
    image_id = annotation["image_id"]

    file_name = ""

    #sram me ove petlje, vjerovatno se moze nekako aproksimirat uz pomoc image_id-a
    for image in images:
        if str(image["id"]) == str(image_id):
            file_name = image["file_name"]
            break

    #u jsonu se nalaze krivi nazivi slika pa je potrebno ispraviti
    paths.append(PATH_TO_IMAGES_FOLDER + "/" + file_name.replace(":","_"))
    labels.append(annotation["category_id"]-1) # lower by one so it starts with zero
    #labels_for_subclasses.append(annotation["category"])

print("Size of paths and labels should be the same: ", len(paths), len(labels), len(labels_for_subclasses))

all_labels = np.array(labels)
print("Labeling should start at zero!")
print("Min label:", all_labels.min())
print("Max label:", all_labels.max())

Size of paths and labels should be the same:  16970 16970 0
Labeling should start at zero!
Min label: 0
Max label: 5


In [6]:
with open("artifacts/paths_labels.json", "w") as f:
    json.dump(
        {"paths": paths, "labels": labels, "labels_for_subclasses": labels_for_subclasses},
        f,
        indent=2
    )